In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import torch
import random


In [2]:


purchase_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/purchase.csv')
boxes_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/boxes.csv')

In [3]:
purchase_df.head()

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0


In [4]:
base_date = datetime.date(2018, 10, 1)
purchase_df['day'] = purchase_df['PURCHASE_DATE'].apply(lambda x: (datetime.datetime.strptime(x, '%d/%m/%Y').date() - base_date).days)



In [5]:
purchase_df.sample(10)

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,day
1424915,24/12/2018,2CD4E969A99,163.0,1.0,84
47626,3/2/2019,2C9C14B6963,162.0,1.0,125
1019603,26/1/2019,28EC65A737C,272.0,1.0,117
2059091,1/10/2018,29047ABEAD2,231.0,1.0,0
1910945,5/11/2018,292EB335891,258.0,1.0,35
285757,15/2/2019,2C0F1E84E30,94.0,1.0,137
104354,6/2/2019,290C1DD791D,255.0,1.0,128
415323,21/2/2019,2C25AAB75C5,28.0,1.0,143
879627,17/1/2019,29662AA5B44,255.0,1.0,108
1512364,29/12/2018,28E4A651E7B,231.0,1.0,89


In [6]:
purchase_df.drop(['PURCHASE_DATE'], axis=1, inplace=True)

In [7]:
customer_keys = purchase_df['MAGIC_KEY'].unique()
customer_ids = {customer_keys[i]: i for i in range(len(customer_keys))}

In [8]:
purchase_df.drop(purchase_df[purchase_df['BOX_ID'].isnull()].index, inplace=True)
purchase_df.drop(purchase_df[purchase_df['BOX_COUNT'] < 0].index, inplace=True)

In [9]:
purchase_df['BOX_COUNT'] = purchase_df['BOX_COUNT'].astype(int)
purchase_df['BOX_ID'] = purchase_df['BOX_ID'].astype(int)

In [10]:
purchase_df.head(10)

,MAGIC_KEY,BOX_ID,BOX_COUNT,day
0,2CED678A247,12,1,123
1,2BF58D91BA1,12,1,123
2,2C15B86534E,99,1,123
3,2C32D9A859A,6,1,123
4,2C7A55404D1,4,1,123
5,29D969045C2,238,1,123
6,28E5EA49074,227,1,123
7,2CEFA3A8659,6,1,123
8,2A00DE30F46,204,1,123
9,291C04B5CBF,231,1,123


In [11]:
len(purchase_df['MAGIC_KEY'].unique())

1274087

In [12]:
len(purchase_df)

2455800

In [13]:
purchases = [[] for _ in range(len(customer_keys))]
purchase_df.apply(lambda x: purchases[customer_ids[x['MAGIC_KEY']]].append((x['BOX_ID'], x['BOX_COUNT'], x['day'])), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
2455859    None
2455860    None
2455861    None
2455862    None
2455863    None
Length: 2455800, dtype: object

In [14]:
prob3_df =pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-3/problem 3.csv')
prob3_df.head()

,MAGIC_KEY
0,2BCFE9C06A7
1,2C2A872B5A2
2,2C6A897671B
3,2C6F1287F53
4,2C658198CC9


In [15]:
len(prob3_df)

5379

In [16]:
prob3_customers = [customer_ids[x] for x in prob3_df['MAGIC_KEY'].unique()]
len(prob3_customers)

5379

In [17]:
for i, _ in enumerate(purchases):
    purchases[i].sort(key=lambda x: x[2])

In [18]:
a = 10
b = 10
print("single purchase: ")
for i, p in enumerate(purchases):
    if len(p) == 1:
        if a > 0:
            print(i, p)
            a -= 1

print("more than 5 purchase: ")
for i, p in enumerate(purchases):
    if len(p) > 5:
        if b > 0:
            print(i, p)
            b -= 1


single purchase: 
0 [(12, 1, 123)]
1 [(12, 1, 123)]
3 [(6, 1, 123)]
4 [(4, 1, 123)]
6 [(227, 1, 123)]
7 [(6, 1, 123)]
10 [(12, 1, 123)]
11 [(231, 1, 123)]
13 [(278, 1, 123)]
16 [(231, 1, 123)]
more than 5 purchase: 
5 [(238, 1, 14), (238, 1, 36), (238, 1, 56), (238, 1, 80), (238, 1, 100), (238, 1, 123)]
14 [(203, 1, 35), (203, 1, 53), (203, 1, 63), (221, 1, 72), (203, 1, 81), (203, 1, 87), (238, 1, 97), (204, 1, 123), (204, 1, 135), (272, 1, 150)]
38 [(8, 1, 15), (8, 1, 30), (8, 1, 60), (8, 1, 82), (8, 1, 104), (8, 1, 123)]
39 [(238, 1, 20), (238, 1, 44), (268, 1, 54), (238, 1, 67), (238, 1, 76), (238, 1, 89), (238, 1, 98), (238, 1, 105), (228, 1, 123), (238, 1, 129), (238, 1, 133), (238, 1, 138), (238, 1, 142), (230, 1, 147), (238, 1, 150)]
42 [(260, 1, 8), (203, 1, 23), (221, 1, 38), (221, 1, 65), (271, 1, 84), (272, 1, 104), (221, 1, 123), (276, 1, 145)]
43 [(137, 1, 6), (137, 1, 23), (137, 1, 42), (137, 1, 62), (137, 1, 79), (174, 1, 97), (174, 1, 123)]
44 [(238, 1, 18), (238, 1, 1

In [19]:
boxes_df.head(10)

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96
2,3,Premium,Home Delivery - CoD,0.0,2.4,11.96
3,4,Premium,Home Delivery - CoD,0.0,2.5,11.96
4,5,Premium,Home Delivery - CoD,0.0,2.6,11.96
5,6,Premium,Home Delivery - CoD,0.0,2.7,11.96
6,7,Premium,Home Delivery - CoD,0.0,2.8,11.96
7,8,Premium,Home Delivery - CoD,10.0,0.0,12.18
8,9,Premium,Home Delivery - CoD,10.5,0.0,12.78
9,10,Premium,Home Delivery - CoD,8.0,1.3,12.98


In [20]:
len(boxes_df)

290

In [21]:
def meat_consumed(cid, start_day, end_day):
    boxes = filter(lambda x: x[2] >= start_day and x[2] < end_day, purchases[cid])
    meat_consumed = 0
    for id, cnt, _ in boxes:
        meat_consumed += cnt * boxes_df.at[id - 1, 'MEAT']
    return meat_consumed

def milk_consumed(cid, start_day, end_day):
    boxes = filter(lambda x: x[2] >= start_day and x[2] < end_day, purchases[cid])
    milk_consumed = 0
    for id, cnt, _ in boxes:
        milk_consumed += cnt * boxes_df.at[id - 1, 'MILK']
    return milk_consumed


def meat_consumption_rate(cid):
    end_day = 151
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MEAT'] > 0.0, purchases[cid]))
    if len(consumptions) == 0:
        return 0.0
    start = consumptions[0][2]
    return meat_consumed(cid, start, end_day) / (end_day - start)

def milk_consumption_rate(cid):
    end_day = 151
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MILK'] > 0.0, purchases[cid]))
    if len(consumptions) == 0:
        return 0.0
    start = consumptions[0][2]
    return milk_consumed(cid, start, end_day) / (end_day - start)

In [22]:
last_day = purchase_df['day'].max()
last_day

150

In [23]:
last_date = datetime.date(2019, 2, 28)
(last_date - base_date).days

150

In [24]:
first_day = purchase_df['day'].min()
first_day

0

In [25]:
meat_consumption_rate(0)

0.05357142857142857

In [26]:
def last_meat_consumption(cid, end_day=151):
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MEAT'] > 0.0, purchases[cid]))
    try:
        bid, cnt, day = consumptions[-1]
    except:
        return None
    return [cnt * boxes_df.at[bid - 1, 'MEAT'], 
            end_day - day]
last_meat_consumption(3588, 150)

[4.4, 27]

In [27]:
split = int(len(purchases) * 0.1)
idx = [i for i in range(len(purchases))]
random.shuffle(idx)
val_idx = idx[:split]
train_idx = idx[split:]

def generate_batch(day, batch_size, split='train'):
    batch = []
    labels = []
    n = 0
    if split == 'train':
        random.shuffle(train_idx)
        current_idx = train_idx
    else:
        random.shuffle(val_idx)
        current_idx = val_idx
    for cid in current_idx:
        if len(purchases[cid]) < 2:
            continue
        last_meat = last_meat_consumption(cid, day)
        if last_meat is not None:
            rate1 = meat_consumption_rate(cid)
            rate2 = milk_consumption_rate(cid)
            next_meat = meat_consumed(cid, day, day + 15)
            if next_meat > 0.0:
                features = [rate1, rate2]
                features.extend(last_meat)
                batch.append(features)
                labels.append(next_meat)
                n += 1
                if split=='train' and n == batch_size:
                    return batch, labels
    if split == 'train':
        return None, None
    else:
        return batch, labels
batch, labels = generate_batch(15, 10)
print(batch)
print(labels)

[[0.06308724832214765, 0.6040268456375839, 1.8, 13], [0.07549668874172184, 0.6357615894039735, 1.8, 15], [0.09051094890510948, 0.5255474452554745, 2.2, 1], [0.13066666666666668, 0.0, 2.7, 14], [0.07516778523489932, 0.6308724832214765, 1.8, 13], [0.07464788732394366, 0.0, 3.6, 6], [0.03776223776223776, 0.2937062937062937, 1.8, 7], [0.07152317880794702, 0.5099337748344371, 1.8, 15], [0.02448979591836735, 0.17006802721088435, 1.8, 11], [0.10948905109489053, 0.0, 2.5, 1]]
[1.8, 1.8, 2.2, 2.5, 1.8, 3.5, 1.8, 1.8, 1.8, 2.4]


In [28]:
import gc

class MeatPredictor(torch.nn.Module):
    def __init__(self):
        super(MeatPredictor, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(4, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1), 
            torch.nn.ReLU()
        )
    def forward(self, x):
        return self.mlp(x).reshape(-1)

model = MeatPredictor().cuda()
model(torch.tensor(batch, dtype=torch.float32).cuda())

tensor([0.3961, 0.4531, 0.0830, 0.4268, 0.3961, 0.2026, 0.2243, 0.4536, 0.3376,
        0.0989], device='cuda:0', grad_fn=<ViewBackward0>)

In [29]:


torch.cuda.empty_cache()
gc.collect()

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
best_loss = 1e9
batch_size = 64
epochs = 2000

val, val_labels = generate_batch(136, split, 'val')
val = torch.tensor(val, dtype=torch.float32).cuda()
val_labels = torch.tensor(val_labels, dtype=torch.float32).cuda()

for epoch in range(epochs):
    batch, labels = generate_batch(random.randint(70, 136), batch_size)
    if batch is None:
        continue
    batch = torch.tensor(batch, dtype=torch.float32).cuda()
    labels = torch.tensor(labels, dtype=torch.float32).cuda()
    optimizer.zero_grad()
    outputs = model(batch)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(epoch, loss.item())
    if epoch % 20 == 0:
        val_outputs = model(val)
        val_loss = loss_fn(val_outputs, val_labels)
        print("Validation loss: ", val_loss.item())
        if val_loss.item() < best_loss:
            torch.save(model.state_dict(), 'meat_predictor.pth')
            best_loss = val_loss.item()

model.load_state_dict(torch.load('meat_predictor.pth'))

0 3.3222928047180176
Validation loss:  3.168585777282715
10 3.992781639099121
20 1.3584660291671753
Validation loss:  2.955327033996582
30 1.1678638458251953
40 1.4165775775909424
Validation loss:  1.4435514211654663
50 0.5132429003715515
60 0.2194729447364807
Validation loss:  0.5872167944908142
70 0.20470163226127625


KeyboardInterrupt: 

In [ ]:
print("best loss: ", best_loss)

In [ ]:
def predict(cid, day):
    rate1 = meat_consumption_rate(cid)
    rate2 = milk_consumption_rate(cid)
    last_meat = last_meat_consumption(cid, day)
    if last_meat is not None:
        features = [rate1, rate2]
        features.extend(last_meat)
        return model(torch.tensor(features, dtype=torch.float32).cuda()).item()
    return 0.0

In [ ]:

data = []
for cid in prob3_customers:
    data.append({
        'MAGIC_KEY': customer_keys[cid],
        'MEAT': predict(cid, 151)
    })
sub_df = pd.DataFrame(data, columns=['MAGIC_KEY', 'MEAT'])
sub_df.to_csv('submission.csv', index=False)